In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt


print(tf.__version__)

2.3.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile

local_zip = '/content/drive/My Drive/Treball de recerca/Datasets/17810_23812_bundle_archive.zip'

zip = zipfile.ZipFile(local_zip, 'r')

zip.extractall('/content')
zip.close()


In [ ]:
!ls /content/chest_xray/test

NORMAL	PNEUMONIA


In [ ]:
import os
dir = '/content/chest_xray'

train_dir = os.path.join(dir, 'train')
train_normal_dir = os.path.join(dir, 'train/NORMAL')
train_pneumonia_dir = os.path.join(dir, 'train/PNEUMONIA')

validation_dir = os.path.join(dir, 'val')
validation_normal_dir = os.path.join(dir, 'val/NORMAL')
validation_pneumonia_dir = os.path.join(dir, 'val/PNEUMONIA')

test_dir = os.path.join(dir, 'test')
test_normal_dir = os.path.join(dir, 'test/NORMAL')
test_pneumonia_dir = os.path.join(dir, 'test/PNEUMONIA')



In [ ]:
print('total training normal images :', len(os.listdir(train_normal_dir ) ))
print('total training pneumonia images :', len(os.listdir(train_pneumonia_dir ) ))

print('total validation normal images :', len(os.listdir( validation_normal_dir ) ))
print('total validation pneumonia images :', len(os.listdir( validation_pneumonia_dir ) ))

print('total test normal images :', len(os.listdir( test_normal_dir ) ))
print('total test pneumonia images :', len(os.listdir( test_pneumonia_dir ) ))


total training normal images : 1341
total training pneumonia images : 3875
total validation normal images : 8
total validation pneumonia images : 8
total test normal images : 234
total test pneumonia images : 390


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( rescale = 1.0/255.  )
validation_datagen  = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(200, 200))     

validation_generator =  validation_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (200, 200))


test_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (200, 200))


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
weight_for_0 = (len(os.listdir(train_normal_dir )) + len(os.listdir(train_pneumonia_dir ))) / (2 * len(os.listdir(train_normal_dir )))
weight_for_1 = (len(os.listdir(train_normal_dir ))+ len(os.listdir(train_pneumonia_dir ))) / (2 * len(os.listdir(train_pneumonia_dir )))

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 1.94
Weight for class 1: 0.67


In [ ]:
import tensorflow as tf
from tensorflow.keras import Model

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(200, 200, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 198, 198, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 46, 46, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 67712)            

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(lr=1e-4),
              loss='binary_crossentropy',
              metrics = ['acc'])


In [ ]:
history = model.fit(
      train_generator,  
      epochs=3,
      validation_data=validation_generator, 
      class_weight=class_weight,
      verbose=1)



Epoch 1/3
261/261 [==============================] - 57s 217ms/step - loss: 0.3517 - acc: 0.7671 - val_loss: 0.4795 - val_acc: 0.8750
Epoch 2/3
261/261 [==============================] - 56s 213ms/step - loss: 0.1195 - acc: 0.9555 - val_loss: 0.2239 - val_acc: 0.8750
Epoch 3/3
261/261 [==============================] - 56s 213ms/step - loss: 0.1033 - acc: 0.9607 - val_loss: 0.1857 - val_acc: 0.9375


In [ ]:
test_lost, test_acc= model.evaluate(test_generator)
print ("Test Accuracy:", test_acc)

32/32 [==============================] - 6s 175ms/step - loss: 0.4319 - acc: 0.8381
Test Accuracy: 0.8381410241127014
